In [1]:
import json
import pandas as pd
#import xlsxwriter
#import glob
import os
from collections import defaultdict
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [2]:
god_set = set(['预言家','女巫','猎人','白痴','守卫','乌鸦','魔术师','骑士','九尾妖狐','禁言长老','潜行者','狐狸','熊',
           '操纵月亮的女孩','小红帽','摄梦人','阴阳使者','守墓人','通灵师','牧师','黑市商人','猎魔人','武僧','小女孩',
            '占卜师','蛊惑师','公爵','锈剑骑士'])
villager_set = set(['平民','混血儿','野孩子'])
wolf_set = set(['狼人','白狼王','黑狼王','梦魇','恶魔','黑蝙蝠','种狼','隐狼','侍僧','狼美人','恶灵骑士','天狗','狼外婆',
                '狐仙','石像鬼','邪恶商人','机械狼','魔狼','狼兄','狼弟','大灰狼','鬼狼','雪狼','灵狼',''])

In [3]:
def update_player_data(ind, json_data):
    name = json_data['name']
    target = player_info[name]['data']
    death_method = json_data['death_method'] if 'death_method' in json_data else '存活'
    if death_method == '猎杀':
        death_method = '中刀'
    if death_method == '毒杀':
        death_method = '中毒'
    if death_method == '枪杀':
        death_method = '中枪'
        
    target.loc[ind] = [
        json_data['seat'],
        json_data['role'],
        json_data['final_camp'],
        death_method,
        '/', 
        '/', 
        '/', 
        '上警' if json_data['votes']['警徽']['target'] == '上警' else '警下',
        '获得警徽' if 'badge' in json_data else '/',
        json_data['result']
    ]

    
def update_player_summary_statistics(name):
    # summary statistics according to chen's excel
    target = player_info[name]
    target_data = target['data']
    target['总局数'] = np.sum(target_data.shape[0])
    target['胜局数'] = np.sum(target_data['胜负'] == '胜利')
    target['负局数'] = np.sum(target_data['胜负'] == '失败')
    target['总胜率'] = target['胜局数'] / target['总局数']
    target['好人胜率'] = np.sum((target_data['阵营'] == '好人') & (target_data['胜负'] == '胜利')) / np.sum(target_data['阵营'] == '好人') 
    target['狼人胜率'] = np.sum((target_data['阵营'] == '狼人') & (target_data['胜负'] == '胜利')) / np.sum(target_data['阵营'] == '狼人') 
    target['存活率'] = np.sum(target_data['死亡'] == '存活') / target['总局数']
    target['放逐率'] = np.sum(target_data['死亡'] == '放逐') / target['总局数']
    target['查验率'] = np.sum(target_data['金水'] != '/') / target['总局数']
    target['中毒率'] = np.sum(target_data['死亡'] == '中毒') / target['总局数']
    target['中刀率'] = np.sum(target_data['死亡'] == '中刀') / target['总局数']
    target['中枪率'] = np.sum(target_data['死亡'] == '中枪') / target['总局数']
    target['自爆率'] = np.sum(target_data['死亡'] == '自爆') / np.sum(target_data['阵营'] == '狼人')
    target['自刀数'] =  np.sum((target_data['阵营'] == '狼人') & (target_data['死亡'] == '中刀')) + \
                        np.sum((target_data['阵营'] == '狼人') & (target_data['银水'] == '银水'))
    target['银水数'] = np.sum((target_data['银水'] == '银水'))
    target['上警率'] = np.sum(target_data['竞选'] == '上警') / target['总局数']
    target['警上好人'] = np.sum((target_data['阵营'] == '好人') & (target_data['竞选'] == '上警')) / np.sum(target_data['竞选'] == '上警')
    target['警下好人'] = np.sum((target_data['阵营'] == '好人') & (target_data['竞选'] == '警下')) / np.sum(target_data['竞选'] == '警下')
    target['警徽率'] = np.sum(target_data['警徽'] == '获得警徽') / target['总局数']
    target['狼人率'] = np.sum(target_data['角色'].isin(wolf_set)) / target['总局数']
    target['平民率'] = np.sum(target_data['角色'].isin(villager_set)) / target['总局数']
    target['神牌率'] = np.sum(target_data['角色'].isin(god_set)) / target['总局数']
    
def parse_json(file = '../werewolf_data/cleaned_data/0829-3.json'):
    with open(file, encoding = 'utf-8') as f:
        data = json.load(f)
    game_id = file[-11:-5]
    
    viewer_id, witch_id, guard_id = 0, 0, 0
    for i in range(1, 13):
        update_player_data(game_id, data['data'][str(i)])
        if data['data'][str(i)]['role'] == '预言家' or data['data'][str(i)]['role'] == '通灵师':
            viewer_id = i
        if data['data'][str(i)]['role'] == '女巫':
            witch_id = i
        if data['data'][str(i)]['role'] == '守卫':
            guard_id = i
            
    # 金水
    if viewer_id:
        viewer_actions = data['data'][str(viewer_id)]['actions']
        for keys, values in viewer_actions.items():
            if values[0]['ability'] == '查验':
                check_id = values[0]['target_seat']
                name = data['data'][str(check_id)]['name']
                target_role = data['data'][str(check_id)]['role']
                check_result = '查杀' if target_role in wolf_set else '金水'
                player_info[name]['data'].loc[game_id, '金水'] = check_result
    
    # 银水
    if witch_id:
        witch_actions = data['data'][str(witch_id)]['actions']
        for keys, values in witch_actions.items():
            if values[0]['ability'] == '解救':
                check_id = values[0]['target_seat']
                name = data['data'][str(check_id)]['name']
                target_role = data['data'][str(check_id)]['role']
                player_info[name]['data'].loc[game_id, '银水'] = '银水'
    
    # 铜水
    if guard_id:
        # TODO
        pass

In [4]:
# unit test
player_info = defaultdict(lambda : {'data': pd.DataFrame(columns = ['号码','角色','阵营','死亡','金水','银水','铜水','竞选','警徽','胜负'])})
parse_json()
player_info['华伋']

{'data':        号码  角色  阵营  死亡  金水 银水 铜水  竞选 警徽  胜负
 0829-3  1  狼人  狼人  自爆  查杀  /  /  上警  /  失败}

In [5]:
player_info = defaultdict(lambda : {'data': pd.DataFrame(columns = ['号码','角色','阵营','死亡','金水','银水','铜水','竞选','警徽','胜负'])})

path_to_json = '../werewolf_data/cleaned_data/'
files = [f for f in os.listdir(path_to_json) if os.path.isfile(path_to_json + f)]
json_files = [json_file for json_file in files if json_file.endswith('.json')]

for json_file in json_files:
    parse_json(path_to_json + json_file)

keys = player_info.keys()
for name in keys:
    update_player_summary_statistics(name)

In [6]:
player_info['华伋']['data']

,号码,角色,阵营,死亡,金水,银水,铜水,竞选,警徽,胜负
0829-1,1,平民,好人,放逐,/,/,/,警下,/,失败
0829-2,1,预言家,好人,中枪,/,/,/,上警,获得警徽,失败
0829-3,1,狼人,狼人,自爆,查杀,/,/,上警,/,失败
0829-4,1,平民,好人,存活,/,/,/,警下,/,胜利
0905-1,1,平民,好人,放逐,/,/,/,警下,/,失败
0905-2,1,平民,好人,存活,金水,/,/,警下,获得警徽,失败
0905-3,1,平民,好人,中刀,金水,/,/,上警,获得警徽,失败
0912-1,1,狼人,狼人,存活,/,/,/,上警,/,胜利
0912-2,1,平民,好人,中刀,金水,/,/,上警,/,失败
0912-3,1,平民,好人,中刀,金水,银水,/,上警,/,胜利


In [7]:
import xlsxwriter

writer = pd.ExcelWriter('release.xlsx', engine='xlsxwriter')
workbook  = writer.book
workbook.formats[0].set_font_size(12)

for name, _ in player_info.items():
    player_info[name]['data'].to_excel(writer, sheet_name= name,
                startrow=12, header=True, index=True)
    #workbook = xlsxwriter.Workbook('release.xlsx')

    worksheet = workbook.get_worksheet_by_name(name)
    worksheet.set_column(0, 100, 10)
    #cell_format = workbook.add_format()
    #cell_format.set_bg_color('orange')
    my_list = [1, 2, 3, 4, 5]
    worksheet.write(0, 0, '玩家姓名：')
    worksheet.write(0, 1, name)

    keys = list(player_info[name].keys())[1:]
    values = np.nan_to_num(list(player_info[name].values())[1:])
    group1 = keys[0:6], values[0:6]
    group2 = keys[7:15], values[7:15]
    group3 = keys[16:25], values[16:25]
    worksheet.write_column(3, 0, group1[0])
    worksheet.write_column(3, 1, group1[1])
    worksheet.write_column(3, 3, group2[0])
    worksheet.write_column(3, 4, group2[1])
    worksheet.write_column(3, 6, group3[0])
    worksheet.write_column(3, 7, group3[1])

writer.save()